In [60]:
import os

In [61]:
%pwd

'/Users/prathmesh/Desktop/chicken_disease-cnnclassifier'

In [62]:
os.chdir("../")

In [63]:
%pwd

'/Users/prathmesh/Desktop'

In [64]:
%cd chicken_disease-cnnclassifier

/Users/prathmesh/Desktop/chicken_disease-cnnclassifier


In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [66]:
from cnnclassifier.constant import *
from cnnclassifier.utils.common import read_yaml,create_directories

In [67]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

        

In [68]:
import os
import urllib.request as request
import zipfile
from cnnclassifier import logger
from cnnclassifier.utils.common import get_size

In [69]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all the files to {unzip_path}")

In [70]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-08-24 10:06:43,408: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-24 10:06:43,410: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-24 10:06:43,410: INFO: common: created directory at: artifacts]
[2025-08-24 10:06:43,411: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-24 10:06:45,805: INFO: 3874346788: artifacts/data_ingestion/data.zip downloaded with following info:
Connection: close
Content-Length: 11621297
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c59eb086f697a4734c2bda47f04f88206df50dfc5d0dcc683113e225dfde848"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7A6D:129AEB:15B89D:3EBE10:68AA96DD
Accept-Ranges: bytes
Date: Sun, 24 Aug 2025 04:36:47 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210023-BOM
X-